# IBCF(Item Based Collaborative Filtering)

### 데이터 준비하기

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
%matplotlib inline

In [5]:
data = pd.read_csv('my_df.csv')
data.head()

,beer,user,rating
0,47658,bluejacket74,4.97
1,66674,bluejacket74,4.04
2,48824,bluejacket74,3.71
3,55939,bluejacket74,4.10
4,68916,bluejacket74,4.34


In [10]:
data['user'].dtype

dtype('O')

In [11]:
data.describe()

,beer,rating
count,101889.000000,101889.000000
mean,27391.059388,4.019884
std,40792.295418,0.495818
min,6.000000,1.000000
25%,1005.000000,3.800000
50%,6301.000000,4.090000
75%,41626.000000,4.330000
max,221289.000000,5.000000


In [12]:
data.isnull().sum()

beer      0
user      0
rating    0
dtype: int64

- shape (101889,3)
- beer&rating: numeric
- user: object
- NaN: None

## preprocessing

In [15]:
ratings_matrix = data.pivot_table('rating', index='user', columns='beer') #ItemID를 기준으로 정렬
ratings_matrix.shape

(333, 481)

In [17]:
ratings_matrix.head()

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
user,,,,,,,,,,,,,,,,,,,,,
57md,3.69,NaN,NaN,NaN,3.63,NaN,NaN,NaN,4.00,2.18,...,3.98,NaN,3.94,3.94,3.69,4.68,3.48,4.18,NaN,NaN
ALC82,NaN,NaN,NaN,NaN,4.50,4.73,NaN,NaN,3.90,2.25,...,NaN,NaN,4.38,NaN,NaN,NaN,NaN,4.25,4.50,NaN
AgentMunky,4.07,3.95,3.16,3.88,4.30,NaN,4.15,NaN,3.46,1.72,...,NaN,4.24,NaN,NaN,NaN,NaN,4.12,NaN,NaN,NaN
Alieniloquium,NaN,4.07,NaN,3.88,4.12,3.95,3.41,NaN,NaN,NaN,...,3.44,NaN,4.12,NaN,NaN,NaN,NaN,NaN,3.95,4.0
Arbitrator,NaN,2.51,3.27,3.56,4.28,4.32,4.01,4.0,4.17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> ratings_matrix.shape = (333,481)

In [18]:
# beerID는 나중에
# rating_beer = pd.merge(data, beers, on='beer_ID')

# columns='beer_ID' 로 변환 & beer_ID 컬럼으로 pivot 수행. 
#ratings_matrix = rating_beer.pivot_table('rating', index='user', columns='beer_ID')

# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
user,,,,,,,,,,,,,,,,,,,,,
57md,3.69,0.00,0.00,0.00,3.63,0.00,0.00,0.0,4.00,2.18,...,3.98,0.00,3.94,3.94,3.69,4.68,3.48,4.18,0.0,0.0
ALC82,0.00,0.00,0.00,0.00,4.50,4.73,0.00,0.0,3.90,2.25,...,0.00,0.00,4.38,0.00,0.00,0.00,0.00,4.25,4.5,0.0
AgentMunky,4.07,3.95,3.16,3.88,4.30,0.00,4.15,0.0,3.46,1.72,...,0.00,4.24,0.00,0.00,0.00,0.00,4.12,0.00,0.0,0.0


### Item간의 유사도 측정
- index가 user로 설정되어있던 ratings matrix를 itemID가 index가 되도록 변환

In [19]:
ratings_matrix_T = ratings_matrix.transpose() #User의 평점이 아니라 영화간의 유사도를 측정하는 프로세스
ratings_matrix_T.head(3)

user,57md,ALC82,AgentMunky,Alieniloquium,Arbitrator,ArrogantB,BARFLYB,BB1313,BEER88,BEERMILER12,...,vfgccp,wedge,weeare138,womencantsail,woodychandler,woosterbill,wordemupg,wvsabbath,zac16125,zeff80
beer,,,,,,,,,,,,,,,,,,,,,
6,3.69,0.0,4.07,0.00,0.00,0.0,0.00,3.6,0.0,0.00,...,0.00,0.00,3.95,0.00,0.0,3.61,0.00,0.00,0.00,3.17
30,0.00,0.0,3.95,4.07,2.51,0.0,3.77,0.0,4.0,4.42,...,3.85,4.79,4.10,0.00,0.0,4.14,4.47,4.38,4.31,4.33
31,0.00,0.0,3.16,0.00,3.27,0.0,0.00,0.0,0.0,3.90,...,4.02,0.00,0.00,3.11,0.0,0.00,4.10,3.06,0.00,4.23


> index가 beer , columns가 user

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도 matrix 작성
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T) #Item유사도 - corr.matrix처럼 n*n행렬

# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,columns=ratings_matrix.columns).round(4) # shape (481,481) # 소수점 4번째 자리까지
item_sim_df.head()

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
beer,,,,,,,,,,,,,,,,,,,,,
6,1.0000,0.7117,0.6496,0.7137,0.7147,0.6112,0.6738,0.6923,0.7124,0.6083,...,0.4869,0.4740,0.4043,0.4582,0.4639,0.4754,0.4922,0.4128,0.2802,0.4054
30,0.7117,1.0000,0.7582,0.9201,0.9364,0.8062,0.8288,0.7936,0.8541,0.7063,...,0.6231,0.5792,0.4821,0.6126,0.5958,0.5943,0.5701,0.5339,0.4125,0.5155
31,0.6496,0.7582,1.0000,0.7624,0.7592,0.6532,0.6952,0.7148,0.7234,0.6334,...,0.4531,0.4725,0.3797,0.4495,0.4679,0.4627,0.4774,0.3830,0.2865,0.3525
33,0.7137,0.9201,0.7624,1.0000,0.9179,0.7924,0.8332,0.8032,0.8376,0.7051,...,0.5939,0.5657,0.4816,0.5787,0.5799,0.5814,0.5504,0.5362,0.3814,0.4999
34,0.7147,0.9364,0.7592,0.9179,1.0000,0.8495,0.8489,0.8123,0.8813,0.7525,...,0.6397,0.5919,0.5468,0.6199,0.6196,0.6463,0.6040,0.5920,0.4511,0.5547


>beer_ID간의 유사도 계산

In [ ]:
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6] #해당 영화와 유사도가 가장 높은 상위 영화 5가지(1등은 자기 자신)

### 유사도가 가장 높은 상위 n가지 추출하는 모듈 만들기
> ※주의: 자기 자신 제외

In [34]:
def corr_top(item,n):
    ans = pd.DataFrame(item_sim_df[item].sort_values(ascending = False)[1:n])
    return ans

In [37]:
corr_top(33,10)

,33
beer,
30,0.9201
34,0.9179
1708,0.9032
88,0.8984
412,0.8900
131,0.8889
1013,0.8871
2512,0.8865
1904,0.8851


## 아이템 기반 인접 이웃 협업 필터링으로 개인화된 영화 추천

In [39]:
### prediction matrix 작성하기 ###

def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

### predict_rating(ratins_arr,item_sim_arr)
> user가 item에 대해 평가한 데이터와 item간의 유사도를 계산한 데이터를 array형식으로 입력받아 rating 예측값을 계산하는 함수
- ratings_arr: user가 item에 대해 점수를 매긴 matrix를 array형식으로 입력
- item_sim_arr: item 유사도를 계산한 matrix를 array형식으로 입력
- ratings_pred: user의 scoring정보와 item유사도를 dot product로 예측값을 연산. array형식으로 반환한다

In [111]:
# 결과 확인하기
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,columns = ratings_matrix.columns)
ratings_pred_matrix.head()

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
user,,,,,,,,,,,,,,,,,,,,,
57md,2.557521,2.539770,2.542699,2.537633,2.537942,2.517111,2.529003,2.544423,2.548360,2.534147,...,2.562583,2.548413,2.502485,2.563750,2.555198,2.511763,2.552204,2.531040,2.415096,2.489413
ALC82,2.264491,2.302495,2.253402,2.297710,2.327306,2.359716,2.313593,2.282406,2.316119,2.328348,...,2.368528,2.329951,2.443169,2.355068,2.342703,2.406483,2.353883,2.408393,2.511186,2.438862
AgentMunky,2.563730,2.579625,2.580783,2.575660,2.572463,2.551211,2.568175,2.565421,2.568891,2.549082,...,2.515182,2.522537,2.468213,2.517621,2.499227,2.498136,2.505825,2.491471,2.425870,2.483437
Alieniloquium,2.025298,2.119282,2.035929,2.109260,2.137561,2.167387,2.133007,2.067375,2.096473,2.077914,...,2.202948,2.135681,2.286017,2.165826,2.133756,2.261145,2.142712,2.233455,2.404158,2.324640
Arbitrator,2.021488,2.059952,2.064185,2.061413,2.054135,2.061562,2.058637,2.055588,2.048894,2.025354,...,1.923869,1.913971,1.912202,1.932263,1.918706,1.926352,1.918871,1.901625,1.875120,1.905469


In [43]:
### mse 출력하기 ###

from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual): ##pred와 actual 모두 array 형태. matrix의 value만 가져오면 됨
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

### get_mse(predm actual)
> 평점이 존재하는 항목만 추려서 모델 성능 평가하기
- pred: 예측값을 담은 prediction matrix를 array형식으로 입력
- actual: 실제값을 array형식으로 입력

<참고>
- array.nonzero(): array에서 0이 아닌 값들의 row, col을 array형식으로 반환 -> 0이 아닌 값의 위치 찾기
- array.flatten(): 2차원의 배열을 1차원으로 한줄서기 시키는 함수
    - 'c'는 순서대로 나열
    - 'f'는 순서 섞기

In [42]:
# 결과보기
print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  2.0673987567209418


> MSE = 2.0674

In [50]:
### 예측 평점(선호도)이 높은 top n개 추천하기 ###

def predict_rating_topsim(ratings_arr, item_sim_arr, n):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape) # (333, 481)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]): ## 481 - beer 개수
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]  ## 유사도 행렬(item_sim_arr)을 오름차순 정렬한 후, 뒤에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]): ## 333 - user 개수
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) ## row번째 user가 가지는 col번째 beer에 대한 예측 평점 = pred[row, col]
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items])) ## 0~5 사이로 범위 맞추기
            
    return pred

### predict_rating_topsim(ratings_arr, item_sim_arr, n)
> 특정 user가 가지는 특정 item(beer)에 대한 예측 평점 구하기
- ratings_arr: 실제 ratings정보를 담은 matrix를 array형식으로 입력
- item_sim_df: 아이템 유사도를 담은 matrix를 array형식으로 입력
- n: 몇 개를 뽑을 건지

<참고>

- np.zeros(shape): 지정한 shape에 맞게 모두 0으로 채워진 array를 생성
- argsort(): array의 전체 elements를 오름차순으로 정렬하는 함수. 단, 배열 안의 값은 values대신 index로 채워져있다.
    - 정렬된 값들로 반환하기 위해서는 array[array.argsort()]로 인덱스 전환을 해야한다.

In [52]:
# 결과 확인하기
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, 20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values )) # top 20개 예측한 matrix와 실제값을 비교하여 성능 확인하기


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,columns = ratings_matrix.columns)

C:\Users\genie\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == '':
C:\Users\genie\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]


아이템 기반 인접 TOP-20 이웃 MSE:  0.3952419924358358


> 아이템 기반 인접 top 20 MSE: 0.3952

In [58]:
user_rating_id = ratings_matrix.iloc[9, :] #9번째 user의 rating정보
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10] #평점을 준 item만 추려서 평점 순으로 나열 - top 10개 출력

beer
1160     5.00
731      5.00
68916    4.94
1703     4.75
131      4.75
47658    4.75
16814    4.74
645      4.70
41815    4.68
10325    4.68
Name: BEERMILER12, dtype: float64

In [92]:
def get_unlist_beer(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 beer정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 beerID를 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.iloc[userId,:] ##shape (481,) - 모든 beer에 대한 Series(평점을 매긴 것+매기지 않은 것 포함)
    
    # user_rating이 0보다 크면 기존에 평점을 매긴 beer임. 대상 index를 추출하여 list 객체로 만듬
    already_beer = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 beerID를 list 객체로 만듬. 
    all_beer_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_beer 해당하는 beer는 all_beer_list 제외함. 
    unlist_beer = [beer for beer in all_beer_list if beer not in already_beer]
    
    return unlist_beer

### get_unlist_beer(ratings_matrix, userId)
> 특정 유저에 대하여 평가 정보가 없는 beer의 리스트를 출력하는 함수
- ratings_matrix: 평점 정보가 있는 matrix (DataFrame형태)
- userId: 특정 유저의 ID

<참고>
- tolist(): list로 변환하는 함수
- iloc(): index기반 DataFrame에 대한 subset 함수
    - 원래 loc기반으로 작성되었으나 userId, beerId 모두 numeric 타입이므로 인덱스 기반의 iloc로 수정

In [96]:
# 결과 확인하기
unlist_beer = get_unlist_beer(ratings_matrix, 9) # 사용자가 평점을 매기지 않은 beer list 추출 ## type int
unlist_str = list(map(str,unlist_beer))

In [112]:
def recomm_beer_by_userid(pred_df, userId, n):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함.
    # 준비
    pred_df.columns = pred_df.columns.astype(str)
    unlist_beer = get_unlist_beer(ratings_matrix, userId) ##입력받은 특정 user가 평점을 매기지 않은 beer list 출력
    
    # loc를 위해 str변환
    user = pred_df.index.tolist()[userId]
    unlist_beer = list(map(str,unlist_beer))
    
    # 최종 
    recomm_beer = pred_df.loc[user,unlist_beer].sort_values(ascending=False)[:n].round(2) ## 예상 평점 내림차순으로 top n개를 추출. 평점은 소수점 둘째까지
    
    return recomm_beer

### recom_beer_by_userid(pred_df, userId, n)
> 예측 평점 데이터를 기반으로 특정 user가 평점을 매기지 않은 beer 중에서 높은 평점으로 예상되는 제품을 n개 추천하기
- pred_df: 모든 user와 모든 beer에 대한 예측 평점 데이터
- userId: target user
- n: 추천할 개수

<참고>
- unlist_beer: target이 평점을 매기지 않은 beer 리스트

<**오류 수정**>
pd.DataFrame.iloc[]인덱스 문제
- user, unlist_beer 모두 int형식이지만 index기반 slicing에 맞지 않은 ID형태
    - (481,333)형태의 인덱스는 0~480, 0~333의 범위 안에 있어야함
- 특정 user의 (원하는)인덱스를 입력하면 userId를 담은 list에서 해당 위치의 userId를 반환->loc의 입력으로
- pred_df의 columns은 각각 beerId를 숫자형식(이름이 아닌)으로 담은 int에 대한 리스트 -> str으로 반환해야함

=> 인덱스 기반의 iloc는 적합하지 않고, userId와 beerId를 str형식으로 변환한 후 loc를 통해 원하는 userId와 unlist_beer를 추출

In [113]:
# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_beer = recomm_beer_by_userid(ratings_pred_matrix, 9, n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_beer = pd.DataFrame(data=recomm_beer.values,index=recomm_beer.index,columns=['pred_score'])
recomm_beer

,pred_score
beer,
387,2.90
236,2.90
1658,2.90
30282,2.90
39286,2.89
73,2.89
361,2.89
402,2.89
3523,2.89


# Surprise 패키지를 이용한 추천 시스템
> surprise.KNNWithMean

In [8]:
import surprise
from surprise import KNNBasic
from surprise import Dataset                                                     
from surprise import Reader                                                      
from surprise import dump
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

reader = surprise.reader.Reader()
data_sur = surprise.Dataset.load_from_df(data,reader)

사용 알고리즘
1. SGD (Stochastic Gradient Descent)
    - reg: 정규화 가중치. 디폴트는 0.02.
    - learning_rate: 최적화 스텝 사이즈. 디폴트는 0.005.
    - n_epochs: 최적화 반복 횟수. 디폴트는 20.
2. ALS (Alternating Least Squares)의 인수
    - reg_i: 상품에 대한 정규화 가중치. 디폴트는 10.
    - reg_u: 사용자에 대한 정규화 가중치. 디폴트는 15.
    - n_epochs: 최적화 반복 횟수. 디폴트는 10.

In [132]:
algo = surprise.BaselineOnly()

np.random.seed(0)
cross_validate(algo, data_sur)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.33839486, 0.34038126, 0.33549609, 0.33660388, 0.33660952]),
 'test_mae': array([0.24237791, 0.24322351, 0.24058372, 0.24007929, 0.24056748]),
 'fit_time': (0.20020794868469238,
  0.20339655876159668,
  0.20307564735412598,
  0.20311594009399414,
  0.21645188331604004),
 'test_time': (0.14975404739379883,
  0.10935068130493164,
  0.24921202659606934,
  0.13941311836242676,
  0.1545560359954834)}

#### MSD 유사도

In [145]:
sim_options = {'name': 'msd'}
algo = surprise.KNNWithMeans(sim_options=sim_options)
cross_validate(algo, data_sur)["test_mae"].mean()

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


0.2367914965344077

#### 코사인 유사도

In [146]:
sim_options = {'name': 'cosine'}
algo = surprise.KNNWithMeans(sim_options=sim_options)
cross_validate(algo, data_sur)["test_mae"].mean()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


0.24067745768545593

#### 피어슨 유사도

In [147]:
sim_options = {'name': 'pearson'}
algo = surprise.KNNWithMeans(sim_options=sim_options)
cross_validate(algo, data_sur)["test_mae"].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.23701753238893414

#### 피어슨-baseline 유사도

In [136]:
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data_sur)["test_mae"].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.28245169688879995

### KNNWithMean

In [137]:
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNWithMeans(sim_options=sim_options)
cross_validate(algo, data_sur)["test_mae"].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.23529248465954233

### 코사인 유사도로 KNN사용하여 예측 모델 구축하기

In [9]:
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [10]:
trainset, testset = train_test_split(data_sur, test_size=.25, random_state=0)

In [12]:
sim_options = {'name': 'cosine'}
algo = surprise.KNNWithMeans(sim_options=sim_options) #KNN 사용하기
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [13]:
predictions = algo.test( testset ) 
print('prediction type :',type(predictions), ' size:',len(predictions)) # 예측값은 list형식의 instance로 반환되며 총 25000개의 length를 가짐
print('prediction 결과의 최초 5개 추출') #상위 5개 목록 출력
predictions[:5]

prediction type : <class 'list'>  size: 25473
prediction 결과의 최초 5개 추출


[Prediction(uid=80197, iid='mig100', r_ui=4.4, est=4.34157425430072, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=6518, iid='BEERMILER12', r_ui=3.88, est=4.064421583682428, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=152542, iid='Beginner2', r_ui=3.85, est=3.7943872572602673, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=580, iid='IckyT2012', r_ui=2.8, est=2.7156434748970293, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=3457, iid='smcolw', r_ui=4.2, est=3.9459584304856, details={'actual_k': 40, 'was_impossible': False})]

In [14]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ] # userID, ItemID, 예상 score - 'mig100' user는 80197번 맥주에 4.34점을 줄 것으로 예측

[(80197, 'mig100', 4.34157425430072),
 (6518, 'BEERMILER12', 4.064421583682428),
 (152542, 'Beginner2', 3.7943872572602673)]

In [15]:
accuracy.rmse(predictions) 

RMSE: 0.3366


0.33664718360906354

### 특정 user가 특정 item에 몇 점을 줄 것인지 예측하기

In [16]:
def pred_user_item(uid,iid):
    uid = str(uid) # 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
    iid = str(iid)
    pred = algo.predict(uid, iid) #아무거나 userID와 ItemID를 입력하면 해당 유저가 해당 영화에 몇점을 줄 것인지 출력하는 프로세스
    
    print(pred)